## Code 1

### import

In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit, leastsq
import scipy.signal as sig

Error: 

### Data

In [ ]:
df = pd.read_csv("G:\MSc Project\Data\hd103891.dat", delimiter='\t')

xData = []
yData=[]
Uncertainty=[]

for row in df[df.columns[0]]:
  xData.append(row)
#print(xData)

for row in df[df.columns[1]]:
  yData.append(row)
#print(yData)

for row in df[df.columns[2]]:
  Uncertainty.append(row)
#print(Uncertainty)

t = np.array(xData) 
vr = np.array(yData) 
jitter = np.array(Uncertainty)

[2453063.80694 2453789.78139 2453790.84262 2453866.60643 2453868.64102
 2454136.82285 2454142.83664 2454234.60119 2454254.52632 2454483.7925
 2454487.83046 2454522.83013 2454540.77333 2454541.78467 2454556.6603
 2454557.65078 2454558.752   2454563.66079 2454582.59476 2454611.57886
 2454813.85368 2454850.82788 2454871.71718 2454872.80015 2454921.78824
 2454930.588   2454931.6658  2455013.52439 2455019.4897  2455019.49746
 2455229.78507 2455237.81571 2455247.8219  2455247.90672 2455266.77331
 2455293.65302 2455325.57564 2455404.47643 2455588.86816 2455672.63399
 2455689.56289 2455695.51915 2455729.47723 2455960.88736 2456023.72922
 2456025.65267 2456051.70645 2456080.47594 2456312.82503 2456332.84591
 2456356.8728  2456383.79655 2456385.71096 2456411.7368  2456672.86097
 2456730.75336 2456731.65786 2456738.74854 2456743.61754 2456763.68259
 2456767.62714 2456819.6071  2456863.47332 2457101.67399 2457147.64728
 2457157.64768 2457211.53054 2457219.52163 2457370.86301 2457373.86733
 2457413

### Radial velocity offset c

In [ ]:

def find_c(t, vr, jitter):
    c = np.mean(vr)
    print("Radial velocity offset (c): ", c)
    return c

c=find_c(t, vr, jitter)

### kepler

In [ ]:
def solve_kep_eqn(M, e):
    E_prev = M
    for i in range(100):
        E_next = M + e*np.sin(E_prev)
        if abs((E_next - E_prev).any()) < 1e-8:
            break
        E_prev = E_next
    return E_next

### Get best n

In [ ]:
def get_best_n(t, vr):
    periods = np.linspace(0.01, 10, 1000)
    frequencies = 2*np.pi/periods
    power = sig.lombscargle(t, vr, frequencies)
    best_n = frequencies[np.argmax(power)]
    return best_n
get_best_n(t, vr)

7.662421106316568

### rv function

In [ ]:
def rv_function(t, K, P, w, e, n):
    n = 2*np.pi/P
    M = n*(t - t[0])
    E = solve_kep_eqn(M, e)
    f = 2*np.arctan2(np.sqrt(1 + e)*np.sin(E/2), np.sqrt(1 - e)*np.cos(E/2))
    return (K*(np.cos(f + w) + e*np.cos(w)))+c

### initial guess

In [ ]:
def get_initial_guesses(t, vr, jitter):
    n0 = get_best_n(t, vr)
    k0 = max(vr)
    #tau0 = t[vr == k0]
    tau0 = t[0]
    w0 = 0
    e0 = 0.5
    periods = 10**np.linspace(0.01,100,1500)
    omega = 2*np.pi/periods
    power = sig.lombscargle(t, vr, omega)
    max_power_index = np.argmax(power)
    p0 = periods[max_power_index]
    initial_guess = (k0, p0, w0, e0, n0)
    return initial_guess
print(get_initial_guesses(t, vr, jitter))

(21.619, 1898.973974138361, 0, 0.5, 7.662421106316568)


### Plot

In [ ]:
ig = get_initial_guesses(t, vr, jitter)
popt, pcov = curve_fit(rv_function, t, vr, ig)

# Extract the fitted parameters from the optimized values
K, P, w, e, n = popt
print(popt)

# Plot the time vs. velocity data and the radial velocity function
plt.plot(t, vr, 'o', label='Data')
plt.plot(t, rv_function(t, K, P, w, e, n), '-', label='Model')
plt.xlabel('Time (days)')
plt.ylabel('Radial Velocity (m/s)')
plt.legend()
plt.show()


## Code 2

#### HD 103891

#### Import Modules

In [ ]:
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import numpy as np
from pylab import*
import pandas as pd

#### Import Data

In [ ]:
data = pd.read_csv("G:\MSc Project\Data\hd103891.dat", sep='\t', header=None)
#, lineterminator='\r'
#print(data)

#### Extract data

In [ ]:
xData = []
yData=[]
Uncertainty=[]

for row in data[data.columns[0]]:
  xData.append(row)
print(xData)

for row in data[data.columns[1]]:
  yData.append(row)
print(yData)

for row in data[data.columns[2]]:
  Uncertainty.append(row)
print(Uncertainty)


[2453063.75727, 2453063.80694, 2453789.78139, 2453790.84262, 2453866.60643, 2453868.64102, 2454136.82285, 2454142.83664, 2454234.60119, 2454254.52632, 2454483.7925, 2454487.83046, 2454522.83013, 2454540.77333, 2454541.78467, 2454556.6603, 2454557.65078, 2454558.752, 2454563.66079, 2454582.59476, 2454611.57886, 2454813.85368, 2454850.82788, 2454871.71718, 2454872.80015, 2454921.78824, 2454930.588, 2454931.6658, 2455013.52439, 2455019.4897, 2455019.49746, 2455229.78507, 2455237.81571, 2455247.8219, 2455247.90672, 2455266.77331, 2455293.65302, 2455325.57564, 2455404.47643, 2455588.86816, 2455672.63399, 2455689.56289, 2455695.51915, 2455729.47723, 2455960.88736, 2456023.72922, 2456025.65267, 2456051.70645, 2456080.47594, 2456312.82503, 2456332.84591, 2456356.8728, 2456383.79655, 2456385.71096, 2456411.7368, 2456672.86097, 2456730.75336, 2456731.65786, 2456738.74854, 2456743.61754, 2456763.68259, 2456767.62714, 2456819.6071, 2456863.47332, 2457101.67399, 2457147.64728, 2457157.64768, 245721

In [ ]:
print(sum(yData)/91)
print((max(yData)-min(yData))/2)

2.122934065934066
25.1665


#### Define Function

In [ ]:
def func(x,k,p,ω,e):
		return (2.13 + (k*np.cos(ω*np.pi/180 + ((2*np.pi*x)/p))) + (k*e*np.cos(ω*100)))

In [ ]:

plt.plot(xData,yData,'ro', label='data points')
initialGuess=(22,1987,170,0.3)

popt,pcov=curve_fit(func,xData,yData,initialGuess,bounds=((18,1800,0,0),(25,2000,360,1000000)), maxfev=5600000) #
print(popt)
xFit=np.arange(2453000,2459000,20)
plt.plot(xFit,func(xFit,*popt),'b',label='fitted curve')

plt.xlabel('Julian date')
plt.ylabel('radial velocity')
plt.legend()
plt.savefig('curvefitting')
plt.show()

In [ ]:
n1=0
sm1=0
for i in xData:
    sq1=i**2
    sm1=sm1+sq1
    n1=n1+1
mean1=sm1/n1
rms_data=mean1**0.5
n2=0
sm2=0
for i in xFit:
    sq2=i**2/10**6
    sm2=sm2+sq2
    n2=n2+1
mean2=sm2/n2
rms_fit=10*3*(mean2**0.5)
sigma=rms_data-rms_fit
print(rms_data, rms_fit, sigma)

2456016.9375120075 90.8779624771597 2455926.0595495305
